## Flow study

In [1]:
import networkx as nx
import numpy as np
import mc_number_paths_estimation as mc_estimation
# Open adjencency list file and build the directed graph
f=open("../lightningAdjList.txt", 'rb')
G=nx.read_multiline_adjlist(f, create_using=nx.Graph)
f.close()

print("Number of nodes: " + str(G.number_of_nodes()))
print("Number of edges: " + str(G.number_of_edges()))

# Read alias file and create a pub_key -> alias dic

Number of nodes: 1647
Number of edges: 8508


In [2]:
A = mc_estimation.init_adjacency(G)

In [5]:
import time

t = time.time()

print(mc_estimation.estimate_number_paths_node(G, 0, iterations=100))


print("Execution took: ", time.time() - t)

{(0, 2): 0.5, (0, 1, 1091, 370, 226, 1227, 10, 897, 642, 26, 79, 946, 25, 473, 35, 395, 101, 41, 214, 5, 713, 20, 60, 317, 71, 272, 12, 623, 7, 598, 57, 592, 752, 27, 635, 63, 56, 58, 496, 55, 420, 277, 373, 113, 51, 393, 88, 143, 176, 17, 96, 99, 106, 308, 820, 506, 13, 168, 153, 331, 42, 76, 111, 360, 472, 75, 990, 919, 1093, 180, 33, 193, 73, 177, 1242, 11, 1261, 59, 231, 637, 825, 591, 1255, 19, 1251): 8.824061383816836e-115, (0, 1, 1279): 0.0020161290322580645, (0, 1, 12, 438, 55, 170, 4, 17, 15, 25, 194, 20, 625, 27, 230, 679, 19, 104, 105, 304, 34, 229, 262, 158, 226, 284, 11, 54, 59, 255, 748, 366, 1075, 5, 210, 71, 562, 63, 1230, 554, 286, 595, 37, 733, 114, 342, 1387, 214, 41, 50, 440, 373, 42, 100, 79, 341, 690, 1033, 33, 46, 111, 966, 143, 73, 360, 13, 244, 84, 152, 74, 368, 937, 224, 7, 986, 196, 922, 10, 934, 482, 354, 501, 709, 35, 804, 26, 823, 338, 1272): 3.982586051486155e-122, (0, 1, 1048, 4, 366, 43, 47, 311, 149, 665, 405, 53, 30, 828, 20, 380, 34, 153, 1022, 1297,